In [2]:
import requests 
import json
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
from pandas import json_normalize
import numpy as np

In [3]:
keylast = os.getenv("keylast")

In [4]:
l_user = 'sinatxester'

# Conectar api lastfm: info usuarios

In [5]:
url_user = f'http://ws.audioscrobbler.com/2.0/?method=user.getinfo&user={l_user}&api_key={keylast}&format=json'

In [9]:
req_user = requests.get(url_user).json()['user']

In [10]:
req_user

{'country': 'Spain',
 'age': '0',
 'playcount': '77520',
 'subscriber': '0',
 'realname': 'Ester Sinatxe',
 'playlists': '0',
 'bootstrap': '0',
 'image': [{'size': 'small',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/daa41927775a61b3e4762c4a8eda80f3.png'},
  {'size': 'medium',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/daa41927775a61b3e4762c4a8eda80f3.png'},
  {'size': 'large',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/daa41927775a61b3e4762c4a8eda80f3.png'},
  {'size': 'extralarge',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/daa41927775a61b3e4762c4a8eda80f3.png'}],
 'registered': {'unixtime': '1420141064', '#text': 1420141064},
 'url': 'https://www.last.fm/user/sinatxester',
 'gender': 'n',
 'name': 'sinatxester',
 'type': 'user'}

In [12]:
req_user.keys()
#nos quedaremos con country, playcount,realname, image[1] (medium), registered['unixtime'],url,gender, name, type
#crearemos tabla en la base de datos de mysql con estos datos

dict_keys(['country', 'age', 'playcount', 'subscriber', 'realname', 'playlists', 'bootstrap', 'image', 'registered', 'url', 'gender', 'name', 'type'])

### Friends del user para conseguir mas amigos

In [33]:
url_fri = f'http://ws.audioscrobbler.com/2.0/?method=user.getfriends&user={l_user}&api_key={keylast}&format=json'

In [36]:
req_fri = requests.get(url_fri).json()['friends']

In [37]:
req_fri.keys()

dict_keys(['@attr', 'user'])

In [38]:
req_fri.keys()


dict_keys(['@attr', 'user'])

In [39]:
req_fri['@attr']

{'perPage': '50',
 'totalPages': '1',
 'page': '1',
 'total': '10',
 'user': 'sinatxester'}

In [40]:
req_fri['user'][0] #lista con todos los amiguismos

{'name': 'Lumanet',
 'url': 'https://www.last.fm/user/Lumanet',
 'country': 'Spain',
 'playlists': '0',
 'playcount': '0',
 'image': [{'size': 'small',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/25602223397211f1f083955ebeacf803.png'},
  {'size': 'medium',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/25602223397211f1f083955ebeacf803.png'},
  {'size': 'large',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/25602223397211f1f083955ebeacf803.png'},
  {'size': 'extralarge',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/25602223397211f1f083955ebeacf803.png'}],
 'registered': {'unixtime': '1196844580', '#text': '2007-12-05 08:49'},
 'realname': 'Marcos Robles',
 'subscriber': '0',
 'bootstrap': '0',
 'type': 'user'}

In [42]:
#para crear una tabla con los amigos, el nombre del usuario mismo es la primary key porque no se puede repetir.
#bueno, primarykey será la unión de nombre user y el id del usuario concatenados
# esta requests tiene paginación con limite... pero para lo que necesitamos no queremos más de 1 pag por usuario.

In [44]:
#iteramos por la lista de amiguismos para observar todas las keys de cada friend
for fri in req_fri['user']:
    print(fri.keys()) #son los datos de usuario que luego se pueden conseguir, por lo que nos quedmso solo con los "name"

dict_keys(['name', 'url', 'country', 'playlists', 'playcount', 'image', 'registered', 'realname', 'subscriber', 'bootstrap', 'type'])
dict_keys(['name', 'url', 'country', 'playlists', 'playcount', 'image', 'registered', 'realname', 'subscriber', 'bootstrap', 'type'])
dict_keys(['name', 'url', 'country', 'playlists', 'playcount', 'image', 'registered', 'realname', 'subscriber', 'bootstrap', 'type'])
dict_keys(['name', 'url', 'country', 'playlists', 'playcount', 'image', 'registered', 'realname', 'subscriber', 'bootstrap', 'type'])
dict_keys(['name', 'url', 'country', 'playlists', 'playcount', 'image', 'registered', 'realname', 'subscriber', 'bootstrap', 'type'])
dict_keys(['name', 'url', 'country', 'playlists', 'playcount', 'image', 'registered', 'realname', 'subscriber', 'bootstrap', 'type'])
dict_keys(['name', 'url', 'country', 'playlists', 'playcount', 'image', 'registered', 'realname', 'subscriber', 'bootstrap', 'type'])
dict_keys(['name', 'url', 'country', 'playlists', 'playcount',

In [48]:
lista_amigos=[]
for fri in req_fri['user']:
    lista_amigos.append(fri['name'].lower())

In [50]:
lista_amigos #para conseguir la infor iríamos iterando por esta lista y llamando a la api get.info de user

['lumanet',
 'trickywagon',
 'crisnuka',
 'ghan64',
 'aiko4985',
 'ljulycastro',
 'rubendomfer',
 'mister__moss',
 'taneli1',
 'avekaesar']

In [51]:
# a la hora de insertarr la info de friends en mysql, primero comprobar si el usuario ya existe. (capitán obvio)

## Conseguir los recenttracks

In [62]:
url_track = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={l_user}&api_key={keylast}&limit=1000&format=json'

In [63]:
req_track = requests.get(url_track).json()['recenttracks']

In [64]:
req_track.keys()

dict_keys(['track', '@attr'])

In [65]:
req_track['@attr'] #aquí podemos observar la paginación... 

{'user': 'sinatxester',
 'totalPages': '78',
 'page': '1',
 'total': '77525',
 'perPage': '1000'}

In [67]:
req_track['track'][0] #puede que el primer elemento sea #nowplaying!! cuidaó con esto 
# el nowplaying está en todas las páginas 

{'artist': {'mbid': '27e948aa-6b38-4da2-acf6-8459964a130a',
  '#text': 'Quique González'},
 'streamable': '0',
 'image': [{'size': 'small',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'medium',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'large',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'extralarge',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png'}],
 'mbid': '',
 'album': {'mbid': '', '#text': 'Pájaros Mojados'},
 'name': 'Avenidas de tu corazón',
 '@attr': {'nowplaying': 'true'},
 'url': 'https://www.last.fm/music/Quique+Gonz%C3%A1lez/_/Avenidas+de+tu+coraz%C3%B3n'}

In [69]:
req_track['track'][1].keys()

dict_keys(['artist', 'streamable', 'image', 'mbid', 'album', 'name', 'url', 'date'])

In [71]:
req_track['track'][1] #nos vamos a quedar solo con una serie de información
#artist: dic (Ambos)
#image: dic size medium. lista. image[1]
#mbid: si no hay da igual, es el mbid del tema (musicbrainz)
#album dic (ambos)
#name 
# url
#date: dic. nos quedamos solo con uts (También tenemos hecha ya una función que nos da el uts)

{'artist': {'mbid': '27e948aa-6b38-4da2-acf6-8459964a130a',
  '#text': 'Quique González'},
 'streamable': '0',
 'image': [{'size': 'small',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'medium',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'large',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'extralarge',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png'}],
 'mbid': '',
 'album': {'mbid': '', '#text': 'Pájaros Mojados'},
 'name': 'Avenidas de tu corazón',
 'url': 'https://www.last.fm/music/Quique+Gonz%C3%A1lez/_/Avenidas+de+tu+coraz%C3%B3n',
 'date': {'uts': '1651051191', '#text': '27 Apr 2022, 09:19'}}

# conseguir info de artista
- una vez que tenemos el nombre y el mbid podemos ir al endpoint del artista
- también con mbid podemos sacar infor del artista de la api de musicbrainz

### endpoint lastfm

In [73]:
artista = req_track['track'][1]['artist']['#text']
art_mbid = req_track['track'][1]['artist']['mbid']
artista,art_mbid

('Quique González', '27e948aa-6b38-4da2-acf6-8459964a130a')

In [74]:
url_art = f'http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artista}&api_key={keylast}&format=json'

In [79]:
req_art = requests.get(url_art).json()['artist']

In [86]:
req_art

{'name': 'Quique González',
 'mbid': '27e948aa-6b38-4da2-acf6-8459964a130a',
 'url': 'https://www.last.fm/music/+noredirect/Quique+Gonz%C3%A1lez',
 'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'small'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'medium'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'large'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'extralarge'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'mega'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': ''}],
 'streamable': '0',
 'ontour': '0',
 'stats': {'listeners': '83313', 'playcount': '2484905'},
 'similar': {'artist': [{'name': 'Ismael Serra

In [87]:
req_art.keys()

dict_keys(['name', 'mbid', 'url', 'image', 'streamable', 'ontour', 'stats', 'similar', 'tags', 'bio'])

In [114]:
# nos vamos a quedar con la sig info
#name, mbid, url, image[1] (medium), #stats(listeners+playcount)
# puede que nos quedemos también con similar y tags.

### get.tag del artista en cuestión

In [88]:
req_art['tags'] #info artista

{'tag': [{'name': 'cantautor', 'url': 'https://www.last.fm/tag/cantautor'},
  {'name': 'spanish', 'url': 'https://www.last.fm/tag/spanish'},
  {'name': 'singer-songwriter',
   'url': 'https://www.last.fm/tag/singer-songwriter'},
  {'name': 'rock', 'url': 'https://www.last.fm/tag/rock'},
  {'name': 'pop', 'url': 'https://www.last.fm/tag/pop'}]}

In [95]:
url_art_tag = f'http://ws.audioscrobbler.com/2.0/?method=artist.gettoptags&artist={artista}&api_key={keylast}&format=json'

In [99]:
req_art_tag = requests.get(url_art_tag).json()['toptags']

In [100]:
req_art_tag.keys()

dict_keys(['tag', '@attr'])

In [102]:
req_art_tag['@attr']

{'artist': 'Quique González'}

In [103]:
req_art_tag['tag'][0] #lista
#cada elemento de la lista es un diccioanrio con count,name y url de keys

{'count': 100, 'name': 'cantautor', 'url': 'https://www.last.fm/tag/cantautor'}

In [107]:
for arttag in req_art_tag['tag']: 
    print(arttag['count'], arttag['name'])

100 cantautor
91 spanish
91 singer-songwriter
89 rock
54 pop
10 indie
7 seen live
7 folk
7 Spanish Rock
7 Cantautores
7 cancion de autor
7 quique gonzalez
5 classic rock
5 country
5 alternative
5 folk-rock
5 spain
5 español
5 male vocalists
5 spanish pop
5 Madrid
3 jazz
3 Grunge
3 pop rock
3 country-rock
3 britpop
3 latin
3 blues
3 singer
3 Alt-country
3 folk rock
3 latino
3 Alternative Folk
3 Rock en Espanol
3 Pop-Rock
3 latin pop
3 spanish indie pop
3 Espana
3 franz ferdinand
3 Bob Dylan
3 All
3 my music
3 lentas
3 Dylan
3 genial
3 spanish indie
3 mediterranean rock
3 Spanish Pop Rock
3 Bunbury
3 canalla
3 patxungo
3 vorgemerkt
3 acustica
3 to listen for jan
3 que genio
3 la vida tiene otro sabor y espanya es la mejor
3 para conocer
3 spanish rock-pop
3 pablo sciuto
3 bilbao live 2008
3 chanson espagnole
3 Qduique Gonzalez


## Búsqueda de artista. Resultados

In [108]:
url_art_bus = f'http://ws.audioscrobbler.com/2.0/?method=artist.search&artist={artista}&api_key={keylast}&format=json'

In [111]:
req_art_bus = requests.get(url_art_bus).json()['results']

In [113]:
req_art_bus

{'opensearch:Query': {'#text': '',
  'role': 'request',
  'searchTerms': 'Quique González',
  'startPage': '1'},
 'opensearch:totalResults': '380',
 'opensearch:startIndex': '0',
 'opensearch:itemsPerPage': '30',
 'artistmatches': {'artist': [{'name': 'Quique González',
    'listeners': '83313',
    'mbid': '27e948aa-6b38-4da2-acf6-8459964a130a',
    'url': 'https://www.last.fm/music/Quique+Gonz%C3%A1lez',
    'streamable': '0',
    'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'small'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'medium'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'large'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'extralarge'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2